source: https://medium.com/onfido-tech/higher-level-apis-in-tensorflow-67bfb602e6c0

# Higher-Level APIs in TensorFlow

![high level api](https://cdn-images-1.medium.com/max/800/1*zoNZvvuJb06yAghetc6BfQ.png)

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.4.1'

## Estimator

The **Estimator** class repressents a model, as well as how this model should be trained and evaluated.

In [4]:
return tf.estimator.Estimator(
    model_fn=model_fn,
    params=params,
    config=run_config)

 - **model_fn** a function to build model.
 - **model_dir** directory to save model paramters.
 - **params** model's hyperparamters should be a [HParams](https://www.tensorflow.org/api_docs/python/tf/contrib/training/HParams) object, which acts as a _namedtuple_
 - **config** specifies how to training and evaluation and run.


### model function

In [5]:
def model_fn(features, labels, mode, params, config):
    """Model function used in the estimator.
    Args:
        features (Tensor): Input features to the model.
        labels (Tensor): Labels tensor for training and evaluation.
        mode (ModeKeys): Specifies if training, evaluation or prediction.
        params (HParams): hyperparameters.
        config ()
    Returns:
        (EstimatorSpec): Model to be run by Estimator.
    """

Follows the signature:
  - **features**: the first item returned from `input_fn`
  - **labels**:   the second item returned from `input_fn`, if `mode` is `ModeKeys.PREDICT`, `labels=None` will passed
  - **model**: an `ModeKeys` object
  - **params** optional `dict` of hyperparamters.
  - **config**
  
return `EstimatorSpec`

**EstimatorSepc** takes in _prediction_, _loss_, _training_ and _evaluation_ operations, and it can defined by `TF-Slim`
  

## Experiment

The **Experiment** class defines how to train a model and integrates with the `Estimator`

In [ ]:
experiment = tf.contrib.learn.Experiment(
    estimator=estimator,
    train_input_fn=train_input_fn,
    eval_input_fn=eval_input_fn,
    
    train_steps=params.train_steps,
    eval_steps=None,
    
    min_eval_frequency=params.min_eval_frequency,
    
    train_monitors=[train_input_hook],
    eval_hooks=[eval_input_hook]
    )

args:
  - **train_input_fn** a python function returns `(features, labels)`
  - **training and evaluating hooks** hooks to help initialize data loder, save model, or monitor training

In [ ]:
# run the experiment
learn_runner.run(
    experiment_fn=experiment_fn,
    run_config=run_config,
    schedule="train_and_evaluate",
    hparams=params
    )

## Dataset

The **Dataset** and **Iterator** help to create data feeders that iterate over the data during training.

In [ ]:
def get_train_input(batch_size, mnist_data):
    
    iterator_initializer_hook = IteratorInitializeHook()
    
    def train_inputs():
        with tf.name_scope('Training_data'):
            images = mnist_data.train.images.reshape([-1, 28, 28, 1])
            labels = mnist.ddata.train.labels
            
            images_placeholder = tf.placehodler(
                images.dtype,
                images.shape)
            labels_placeholder = tf.placeholder(
                labels.dtype,
                labels.shape)
            
            # Build dataset Iterator
            dataset = tf.contrib.data.Dataset.from_tensor_slice(
                (images_placeholder, labels_placeholder))
            dataset = dataset.repeat(None) # Infinite iterations
            dataset = dataset.shuffle(buffer_size=10000)
            dataset = dataset.batch(batch_size)
            iterator = dataset.make_initializable_iterator()
            
            next_example, next_label = iterator.get_next()
            
            iterator_initializer_hook.iterator_initializer_func = \
                lambda sess: sess.run(
                    iterator.initializer,
                    feed_dict={images_placeholder: images,
                               labels_placeholder: labels})
                
            return next_example, next_label
        
    return train_inputs, iterator_initializer_hook

  - create ad sliced dataset by `from_tesnor_slices((features, labels))`
  - need to initialize the placeholder before use


In [4]:
class IteratorInitializerHook(tf.train.SessionRunHook):
    
    def __init__(self):
        super().__init__()
        self.iterator_initializer_func = None
        
    def after_create_session(self, session, coord):
        """Initialize the iterator after the session has been created."""
        self.iterator_initializer_func(session)

We create a custom defined `IteratorInitializerHook` object to initialize the iterator when the graph is created.

## Complete example

In [7]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data as mnist_data
from tensorflow.contrib import slim
from tensorflow.contrib.learn import ModeKeys
from tensorflow.contrib.learn import learn_runner

In [8]:
# show debugging output
tf.logging.set_verbosity(tf.logging.DEBUG)

In [11]:
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string(
    flag_name='model_dir',
    default_value='./dataset/mnist_training',
    docstring='Output directory for model and training stats.')

tf.app.flags.DEFINE_string(
    flag_name='data_dir',
    default_value='./dataset/mnist_data',
    docstring='Directory to download the data to.')

In [12]:
#### Define and run experiment
def run_experiment(argv=None):
    """Run the training experiment"""
    params = tf.contrib.training.HParams(learning_rate=2e-3,
                                         n_classes=10,
                                         train_steps=5000,
                                         min_eval_frequency=100)
    
    # Set the run_config and the directory to save the model and stats
    run_config = tf.contrib.learn.RunConfig()
    run_config = run_config.replace(model_dir=FLAGS.model_dir)
    
    learn_runner.run(experiment_fn=experiment_fn,
                     run_config=run_config,
                     schedule="train_and_evaluate",
                     hparams=params)
    

In [13]:
def experiment_fn(run_config, params):
    """Create an experiment to train and evaluate the model."""
    
    run_config = run_config.replace(
        save_checkpoints_steps=params.min_eval_frequency)
    
    estimator = get_estimator(run_config, params)
    
    # Setup data loaders
    mnist = mnist_data.read_data_sets(FLAGS.data_dir, one_hot=False)
    train_input_fn, train_input_hook = get_train_inputs(
        batch_size=128, mnist_data=mnist)
    eval_input_fn, eval_input_hook = get_test_inputs(
        batch_size=128, mnist_data=mnist)
    
    # Define the experiment
    experiment = tf.contrib.learn.Experiment(
        estimator=estimator,
        train_input_fn=train_input_fn,
        eval_input_fn=eval_input_fn,
        train_steps=params.train_steps,
        min_eval_frequency=params.min_eval_frequency,
        train_monitors=[train_input_hook],
        eval_hooks=[eval_input_hook],
        eval_steps=None)
    
    return experiment
    

In [14]:
def get_estimator(run_config, params):
    """Return the model as a Tensorflow Estimator object."""
    
    return tf.estimator.Estimator(
        model_fn=model_fn,
        params=params,
        config=run_config)

In [30]:
def model_fn(features, labels, mode, params):
    
    is_training = mode == ModeKeys.TRAIN
    
    # model's architecture
    logits = architecture(features, is_training=is_training)
    predictions = tf.argmax(logits, axis=-1)
    
    # Loss, training and eval operations are not needed during inference.
    loss = None
    train_op = None
    eval_metric_ops = {}
    
    if mode != ModeKeys.INFER:
        loss = tf.losses.sparse_softmax_cross_entropy(labels=tf.cast(labels, tf.int32),
                                                      logits=logits)
        train_op = get_train_op_fn(loss, params)
        eval_metric_ops = get_eval_metric_ops(labels, predictions)
        
    return tf.estimator.EstimatorSpec(mode=mode,
                                      predictions=predictions,
                                      loss=loss,
                                      train_op=train_op,
                                      eval_metric_ops=eval_metric_ops)
    
    

In [16]:
def get_train_op_fn(loss, params):
    """ Get the training op."""
    return tf.contrib.layers.optimize_loss(loss=loss,
                                           global_step=tf.contrib.framework.get_global_step(),
                                           optimizer=tf.train.AdamOptimizer,
                                           learning_rate=params.learning_rate)

def get_eval_metric_ops(labels, predictions):
    """Return a dict to the evaluate Ops"""
    return {'Accuracy': tf.metrics.accuracy(labels=labels,
                                            predictions=predictions,
                                            name='accuracy')}
    

In [26]:
def architecture(inputs, is_training, scope='MnistConvNet'):
    with tf.variable_scope(scope):
        with slim.arg_scope([slim.conv2d, slim.fully_connected],
                            weights_initializer=tf.contrib.layers.xavier_initializer()):
            net = slim.conv2d(inputs, 20, [5, 5], padding='VALID', scope='conv1')
            net = slim.max_pool2d(net, 2, stride=2, scope='pool2')
            net = slim.conv2d(net, 40, [5, 5], padding='VALID', scope='conv3')
            net = slim.max_pool2d(net, 2, stride=2, scope='pool4')
            
            net = tf.reshape(net, [-1, 4 * 4 * 40])
            
            net = slim.fully_connected(net, 256, scope='fn5')
            net = slim.dropout(net, is_training=is_training, scope='dropout5')
            net = slim.fully_connected(net, 256, scope='fn6')
            net = slim.dropout(net, is_training=is_training, scope='dropout6')
            
            net = slim.fully_connected(net, 10, scope='output', activation_fn=None)
    return net
            

In [32]:
##### Define data loaders

class IteratorInitializerHook(tf.train.SessionRunHook):
    
    def __init__(self):
        super().__init__()
        self.iterator_initializer_func = None
    
    def after_create_session(self, session, coord):
        self.iterator_initializer_func(session)
        
# Define the training inputs
def get_train_inputs(batch_size, mnist_data):
    
    iterator_initializer_hook = IteratorInitializerHook()
    
    def train_inputs():
        
        with tf.name_scope('Training_data'):
            
            # get mnist data
            images = mnist_data.train.images.reshape([-1, 28, 28, 1])
            labels = mnist_data.train.labels
            
            images_placeholder = tf.placeholder(images.dtype, images.shape)
            labels_placeholder = tf.placeholder(labels.dtype, labels.shape)
            
            dataset = tf.contrib.data.Dataset.from_tensor_slices(
                (images_placeholder, labels_placeholder))
            
            dataset = dataset.repeat(None)
            dataset = dataset.shuffle(buffer_size=10000)
            dataset = dataset.batch(batch_size)
            iterator = dataset.make_initializable_iterator()
            next_example, next_label = iterator.get_next()
            
            # Set runhook to initialize iterator
            
            iterator_initializer_hook.iterator_initializer_func = \
                lambda sess: sess.run(iterator.initializer,
                                      feed_dict={images_placeholder: images,
                                                 labels_placeholder: labels})
                
            return next_example, next_label
        
    return train_inputs, iterator_initializer_hook

def get_test_inputs(batch_size, mnist_data):
    
    iterator_initializer_hook = IteratorInitializerHook()
    
    def test_inputs():
        
        with tf.name_scope("Test_data"):
            images = mnist_data.test.images.reshape([-1, 28, 28, 1])
            labels = mnist_data.test.labels
            
            images_placeholder = tf.placeholder(images.dtype, images.shape)
            labels_placeholder = tf.placeholder(labels.dtype, labels.shape)
            
            # build dataset iterator
            dataset = tf.contrib.data.Dataset.from_tensor_slices(
                (images_placeholder, labels_placeholder))
            dataset = dataset.batch(batch_size)
            iterator = dataset.make_initializable_iterator()
            
            next_example, next_label = iterator.get_next()
            
            iterator_initializer_hook.iterator_initializer_func = \
                lambda sess: sess.run(iterator.initializer,
                                      feed_dict={images_placeholder: images,
                                                 labels_placeholder: labels})
            return next_example, next_label
        
    return test_inputs, iterator_initializer_hook

In [33]:
tf.app.run(main=run_experiment)

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a4dcde1d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './dataset/mnist_training'}
Extracting ./dataset/mnist_data/train-images-idx3-ubyte.gz
Extracting ./dataset/mnist_data/train-labels-idx1-ubyte.gz
Extracting ./dataset/mnist_data/t10k-images-idx3-ubyte.gz
Extracting ./dataset/mnist_data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./dataset/mnist_training/model.ckpt

INFO:tensorflow:Saving checkpoints for 1202 into ./dataset/mnist_training/model.ckpt.
INFO:tensorflow:global_step/sec: 40.8163
INFO:tensorflow:loss = 0.127009, step = 1202 (2.450 sec)
INFO:tensorflow:Starting evaluation at 2017-12-10-11:50:24
INFO:tensorflow:Restoring parameters from ./dataset/mnist_training/model.ckpt-1202
INFO:tensorflow:Finished evaluation at 2017-12-10-11:50:25
INFO:tensorflow:Saving dict for global step 1202: Accuracy = 0.9904, global_step = 1202, loss = 0.0307003
INFO:tensorflow:Validation (step 1300): Accuracy = 0.9904, loss = 0.0307003, global_step = 1202
INFO:tensorflow:Saving checkpoints for 1302 into ./dataset/mnist_training/model.ckpt.
INFO:tensorflow:global_step/sec: 38.0982
INFO:tensorflow:loss = 0.0198861, step = 1302 (2.625 sec)
INFO:tensorflow:Starting evaluation at 2017-12-10-11:50:26


KeyboardInterrupt: 